In [ ]:
import os
import pandas
import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore")

datadir = '/home/idies/workspace/raddick_census/pop2016'
os.chdir(datadir)

os.getcwd()

In [ ]:
counties = pandas.read_csv('co-est2016-alldata.csv', encoding='utf-8')
counties = counties[counties['SUMLEV'] == 50]
counties['GEOID'] = counties['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' +  counties['STATE'].apply(lambda x: '{:02d}'.format(x)) + counties['COUNTY'].apply(lambda x: '{:03d}'.format(x)) 
counties = counties.set_index('GEOID')

counties_out = pandas.DataFrame()
counties_out = counties_out.append(counties[(counties['STATE'] == 6) & (counties['COUNTY'].apply(lambda x: x in [37, 59, 71]))])
counties_out = counties_out.append(counties[(counties['STATE'] == 34) & (counties['COUNTY'] == 5)])
counties_out = counties_out.append(counties[(counties['STATE'] == 36) & (counties['COUNTY'] == 119)])
counties_out = counties_out.append(counties[(counties['STATE'] == 17) & (counties['COUNTY'].apply(lambda x: x in [31, 43, 89, 93, 197]))])
counties = counties.drop(counties_out.index, axis=0)


subdivs = pandas.read_csv('sub-est2016_all.csv', encoding='utf-8')

# California (state 6): Los Angeles (37), Orange (59), San Bernardino (71)
# New Jersey (state 34): Burlington (5)
# New York (state 36): Westchester (19)
# Illinois (state 17): Cook (31), DuPage (43), Kane (89), Kendall (93), Will (197)

ca = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 6) & (subdivs['COUNTY'].apply(lambda x: x in [37, 59, 71]))]
ca['GEOID'] = ''

ca['GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '06US' + ca['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

ca.loc[ca['PLACE'] == 99990, 'GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '00US' + ca['STATE'].apply(lambda x: '{:02d}'.format(x)).astype('str') + ca['COUNTY'].apply(lambda x: '{:02d}'.format(x)).astype('str') + 'x'
#ca['GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '6' + ca['COUNTY'].apply(lambda x: '{:02d}'.format(x)).astype('str') + 'U' + ca['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

#ca.loc[(ca['SUMLEV'] == 157) & (ca['FUNCSTAT'] == 'F'), 'GEOID'] = ca['SUMLEV'].apply(lambda x: '{:03d}'.format(x)) + '00US' + subdivs['STATE'].apply(lambda x: '{:02d}'.format(x)) + ca['COUNTY'].apply(lambda x: '{:03d}'.format(x)) + 'xx'
ca = ca.set_index('GEOID')

nj = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 34) & (subdivs['COUNTY'].apply(lambda x: x in [5]))]
nj['GEOID'] = nj['SUMLEV'].apply(lambda x: '{:02d}'.format(x)).astype('str') + '345US' + nj['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
nj = nj.set_index('GEOID')

ny = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 36) & (subdivs['COUNTY'].apply(lambda x: x in [119]))]
ny['GEOID'] = ny['SUMLEV'].apply(lambda x: '{:02d}'.format(x)).astype('str') + '36119' + ny['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
ny = ny.set_index('GEOID')

il = subdivs[(subdivs['PRIMGEO_FLAG'] == 1) & (subdivs['STATE'] == 17) & (subdivs['COUNTY'].apply(lambda x: x in [31, 43, 89, 93, 197]))]

il['GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17PL' + il['PLACE'].apply(lambda x: '{:05d}'.format(x)).astype('str')

il.loc[(il['PLACE'] == 99990), 'GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17CS' + il['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
il.loc[(il['SUMLEV'] == 61), 'GEOID'] = il['SUMLEV'].apply(lambda x: '{:03d}'.format(x)).astype('str') + '17CS' + il['COUSUB'].apply(lambda x: '{:05d}'.format(x)).astype('str')
il = il.set_index('GEOID')


df = pandas.DataFrame()
df = df.append(counties)
df = df.append(ca)
df = df.append(nj)
df = df.append(ny)
df = df.append(il)

#df.sample(20)
show_columns = ['SUMLEV', 'STATE', 'COUNTY', 'STNAME']
show_columns += ['CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010']
show_columns += ['POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013']
show_columns += ['POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016']

new = pandas.read_csv('newstates.csv', dtype={'GEOID': object, 'state_new': object}, low_memory=False)
new = new.set_index('GEOID')

df = df.join(new, how='left')
#df[show_columns]

df['NAME'] = df['NAME'].fillna(df['CTYNAME'])

df.loc[df['CENSUS2010POP'] == 'A', 'CENSUS2010POP'] = 0
df['CENSUS2010POP'] = pandas.to_numeric(df['CENSUS2010POP'])
df['change_since_2010'] = df['POPESTIMATE2016'] - df['CENSUS2010POP']
#df['percent_change_since_2010'] = ((df['POPESTIMATE2016'] - df['CENSUS2010POP']) / df['CENSUS2010POP'])

#show_columns = ['NAME', 'POPESTIMATE2016', 'STATE', 'SUMLEV', 'state_new', 'percent_change_since_2010']

df.groupby('state_new')[['CENSUS2010POP', 'POPESTIMATE2016', 'change_since_2010']].sum()

In [ ]:
df[show_columns].sample(10)


In [ ]:
df['NAME'] = df['NAME'].fillna(df['CTYNAME'])

df.loc[df['CENSUS2010POP'] == 'A', 'CENSUS2010POP'] = 0
df['CENSUS2010POP'] = pandas.to_numeric(df['CENSUS2010POP'])
df['change_since_2010'] = df['POPESTIMATE2016'] - df['CENSUS2010POP']



show_columns = ['NAME', 'POPESTIMATE2016', 'STATE', 'SUMLEV', 'state_new', 'change_since_2010']

df[show_columns].sample(10)

In [ ]:

df[show_columns]